In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/My Drive/wnut')

In [3]:
os.getcwd()

'/content/drive/My Drive/wnut'

In [10]:
# !pip install torch==1.6.0+cu92 torchvision==0.7.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install allennlp
# !pip install flair
# !pip install tweet-preprocessor



/bin/bash: conda: command not found


In [5]:
import flair, torch

device = None
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

flair.device = device

In [6]:
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from flair.embeddings import StackedEmbeddings, DocumentLSTMEmbeddings
from flair.embeddings import FlairEmbeddings, TransformerWordEmbeddings, ELMoEmbeddings
from flair.training_utils import EvaluationMetric


In [7]:
data_folder = 'data'

# column format indicating which columns hold the text and label(s)
column_name_map = {0: "id", 1: "text", 2: "label"}

# load corpus containing training, test and dev data and if CSV has a header, you can skip it
corpus = CSVClassificationCorpus(data_folder, column_name_map, skip_header=True, delimiter='\t')

2020-08-09 03:14:18,740 Reading data from data
2020-08-09 03:14:18,740 Train: data/train.csv
2020-08-09 03:14:18,743 Dev: data/dev.csv
2020-08-09 03:14:18,744 Test: data/test.csv


In [8]:
elmo_embedding = ELMoEmbeddings('small')

2020-08-09 03:14:24,941 ----------------------------------------------------------------------------------------------------
2020-08-09 03:14:24,942 ATTENTION! The library "allennlp" is not installed!
2020-08-09 03:14:24,942 To use ELMoEmbeddings, please first install with "pip install allennlp==0.9.0"
2020-08-09 03:14:24,947 ----------------------------------------------------------------------------------------------------


UnboundLocalError: ignored

In [ ]:
document_embeddings = DocumentLSTMEmbeddings([elmo_embedding], hidden_size=512, reproject_words=True, reproject_words_dimension=256)


In [ ]:
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)

trainer = ModelTrainer(classifier, corpus)


In [ ]:
trainer.train('elmo-medium-model/', 
              max_epochs=2, mini_batch_size=32,
              embeddings_storage_mode='gpu')


In [ ]:
classifier = TextClassifier.load('elmo-medium-model/best-model.pt')
classifier.embeddings.cpu()

In [ ]:
print(classifier)

In [ ]:
from flair.data import Sentence
import pandas as pd



y_true, y_pred = [], []
for idx in pd.read_csv('data/test.csv', sep='\t', header=None)[1]:
    pred = classifier.predict(Sentence(idx))
    print(pred)
    # y_pred.append(pred)
    # y_true.append(corpus.test[idx].labels)
    # print(pred)
    # corpus.test[idx].labels

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, digits=4))